# Pre-process Sleep Data

This script loads and pre-processes the sleep polysomnography biosignals from the Cleveland Family Study (https://sleepdata.org/datasets/cfs). We then filter the EEG, EMG, EOG, and ECG signals, re-reference the EEG data to the linked mastoids, and then extract our epochs as an MNE object. The epochs of interest are 4 seconds in length and do not overlap. We then optionally downsample everything to 256 Hz and save the epochs as ".fif.gz" files and the hypnograms as ".npy" files.

In [1]:
%matplotlib inline

## Import packages 
import numpy as np
import yasa
import os
import mne
from tqdm import tqdm
import random
import neurokit2 as nk
import pandas as pd
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from scipy.stats import mode
mne.set_log_level('WARNING')

In [2]:
## Additonal useful functions

def process_raw_EDF_cfs(file):
    """
    Process a raw EDF file, apply various preprocessing steps, and return the processed raw data, epochs, and hypnogram.

    This function:
    - Imports the raw EDF file.
    - Maps and picks channels of interest.
    - Rereferences EEG data.
    - Bipolarizes EOG and EMG data.
    - Applies various filters.
    - Epochs the data into fixed-length 4-second segments.
    - Downsamples the data.
    - Imports and unravels the hypnogram.

    Parameters:
    - file (str): Path to the EDF file (without the '.edf' extension).

    Returns:
    - tuple: A tuple containing:
        - raw_train (mne.io.Raw): Processed raw data.
        - epochs (mne.Epochs): Epochs created from the raw data.
        - hypnogram (numpy array): An array representing the hypnogram.

    Raises:
    - ValueError: If there's a mismatch between the hypnogram length and the total number of epochs.
    """

    ## Import raw edf file
    raw_train = mne.io.read_raw_edf(file + '.edf', eog = ['LOC','ROC'],
                                    preload = True, verbose = False)

    ## Create dictionary of channels we are interested in  
    mapping = {'C3': 'eeg',
               'C4': 'eeg',
               'M1': 'eeg',
               'M2': 'eeg',
               'LOC': 'eog',
               'ROC': 'eog',
               'EMG2': 'emg',
               'EMG3': 'emg',
               'ECG1': 'ecg'}

    ## Select channels in object and give labels for channel type
    raw_train.pick_channels(ch_names=list(mapping))
    raw_train.set_channel_types(mapping) 

    ## Rereference eeg data to average of mastoids
    raw_train.set_eeg_reference(ref_channels=['M1','M2']) # type: ignore
        
    ## Bipolarize eog and emg data 
    try:
        raw_train = mne.set_bipolar_reference(raw_train, 'EMG2', 'EMG3')
    except:
        if not isinstance(raw_train, mne.io.Raw):
            ref_inst = mne.io.RawArray(raw_train.get_data(), raw_train.info)
            raw_train = mne.set_bipolar_reference(raw_train, 'EMG2', 'EMG3')

    ## Filter data
    raw_train.filter(picks=['eeg','eog'], l_freq=0.5, h_freq=45)
    raw_train.filter(picks='emg', l_freq=10, h_freq=100)
    # Clean the ECG data with neurokit2
    raw_train.apply_function(fun=nk.ecg_clean, picks='ecg', n_jobs=-1, 
                            channel_wise=True, **dict(sampling_rate=raw_train.info['sfreq'], method='neurokit', powerline=60))
    # Notch filter
    raw_train.notch_filter(freqs=[60, 120], method='spectrum_fit') # type: ignore

    ## Create fixed-length 4 second epochs
    events = mne.make_fixed_length_events(raw_train, duration=4)
    epochs = mne.Epochs(raw_train, events, tmin=0, tmax=3.99,
                        baseline=None, detrend=None, preload=True, reject=None)

    ## Downsample data
    epochs.resample(256)
    raw_train.resample(256)

    ## import and unravel hypnogram
    stages, stagelens = read_xml(file + '-nsrr.xml')
    hypnogram = unravel_hypnogram(stages, stagelens)
  
    return raw_train, epochs, hypnogram

def read_xml(file):
    """
    Reads an XML annotation file to extract hypnogram information.

    Args:
    - file (str): Path to the XML file.

    Returns:
    - tuple: Two numpy arrays containing stages and their corresponding lengths.
    """
    
    tree = ET.parse(file)
    root = tree.getroot()

    # Use list comprehensions to extract the relevant data
    stages = [int(child[1].text[-1]) for child in root.iter('ScoredEvent') 
              if child[0].text and 'Stages' in child[0].text]
    stagelens = [int(float(child[3].text) / 30) for child in root.iter('ScoredEvent') 
                 if child[0].text and 'Stages' in child[0].text]

    # Convert lists to numpy arrays
    return np.array(stages, dtype=int), np.array(stagelens, dtype=int)

def unravel_hypnogram(stages, stagelens):
    """
    Construct a hypnogram based on provided sleep stages and their durations.

    The function maps each sleep stage to a respective value and then creates
    a continuous array representing the hypnogram.

    Parameters:
    - stages (list or array-like): A list of sleep stages, where each stage is an integer.
    - stagelens (list or array-like): A list of durations (in 30s increments) corresponding to each sleep stage.

    Returns:
    - numpy array: A continuous array representing the hypnogram.

    Raises:
    - ValueError: If the length of the constructed hypnogram does not match the total duration specified by stagelens.
    """

    # Map stages to their respective values
    stage_map = {
        0: 0,
        1: 1,
        2: 2,
        3: 3,
        4: 3,  # collapse stage 3 and 4
        5: 4
    }

    # Construct the hypnogram using list comprehension and the mapping
    hypnogram = np.concatenate([stage_map[stage] * np.ones(length) for stage, length in zip(stages, stagelens)])
    
    # Sanity check
    if len(hypnogram) != stagelens.sum():
        raise ValueError('The length of the scaled hypnogram does not match the amount of total epochs')
    
    return hypnogram

def downsample_hypnogram(hypno, data, sf=None, epoch_len_sec=4):
    """Downsample hypnogram to fit epoch length of data.

    Parameters
    ----------
    hypno : array_like
        The sleep staging (hypnogram) 1D array.
    data : np.array_like or mne.io.Raw
        1D or 2D EEG data. Can also be a MNE Raw object, in which case data and sf will be
        automatically extracted.
    sf : float, optional
        The sampling frequency of data AND the hypnogram.
    epoch_len_sec : int
        Length of each epoch in seconds.

    Returns
    -------
    hypno_ds : array_like
        Downsampled hypnogram, with one stage per epoch.
    """
    # Check if data is an MNE raw object
    if isinstance(data, mne.io.BaseRaw):
        sf = data.info["sfreq"]
        data = data.times  # 1D array and does not require to preload data
    data = np.asarray(data)
    hypno = np.asarray(hypno)
    assert hypno.ndim == 1, "Hypno must be 1D."

    # Calculate the number of data points per epoch
    npts_per_epoch = int(epoch_len_sec * sf) # type: ignore

    # Number of epochs in the data
    n_epochs = data.shape[-1] // npts_per_epoch

    # Downsample hypnogram by taking the mode within each epoch
    hypno_ds = []
    for i in range(n_epochs):
        epoch_mode = mode(hypno[i*npts_per_epoch:(i+1)*npts_per_epoch], keepdims=False)[0]
        if np.isscalar(epoch_mode):
            hypno_ds.append(epoch_mode)
        else:
            hypno_ds.append(epoch_mode[0])

    return np.array(hypno_ds)


## 1. Load data

In [3]:
path = '/media/administrator/data/cfs/polysomnography/'
save_path = '/mnt/server/data03/2023_NENA_Aperiodic_Workshop/data/processed/'
fig_path = '/mnt/server/data03/2023_NENA_Aperiodic_Workshop/figures/subject/'
# Iterate over all files if the names end in .edf
files = [os.path.splitext(f)[0] for f in os.listdir(path) if f.endswith('.edf')]
# Randomly select 50 files 
files = random.sample(files, 50)

## 2. Process and save data

In [4]:
# Iterate over all files and process them
for idx, file in enumerate(tqdm(files)):
    print(f'Preprocessing file : {file}')
    raw, epochs, hypnogram = process_raw_EDF_cfs(path + file)
    # save the data
    epochs.save(fname = save_path + file + '-epo.fif.gz', 
                overwrite=False)
    raw.save(fname = save_path + file + '-raw.fif.gz',  # type: ignore
             overwrite=False)
    np.save(save_path + file + '-hypnogram.npy', hypnogram)
    # delete the data from memory
    del raw, epochs, hypnogram

  0%|          | 0/50 [00:00<?, ?it/s]

Preprocessing file : cfs-visit5-801662


/tmp/ipykernel_34950/180965475.py:46: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw_train.pick_channels(ch_names=list(mapping))
  2%|▏         | 1/50 [01:21<1:06:28, 81.39s/it]

Preprocessing file : cfs-visit5-802709


/tmp/ipykernel_34950/180965475.py:46: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw_train.pick_channels(ch_names=list(mapping))
  4%|▍         | 2/50 [02:54<1:10:50, 88.54s/it]

...

/tmp/ipykernel_34950/180965475.py:46: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw_train.pick_channels(ch_names=list(mapping))
 96%|█████████▌| 48/50 [1:38:02<03:44, 112.40s/it]

Preprocessing file : cfs-visit5-801196


/tmp/ipykernel_34950/180965475.py:46: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw_train.pick_channels(ch_names=list(mapping))
 98%|█████████▊| 49/50 [1:40:06<01:55, 115.97s/it]

Preprocessing file : cfs-visit5-802073


/tmp/ipykernel_34950/180965475.py:46: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw_train.pick_channels(ch_names=list(mapping))
100%|██████████| 50/50 [1:41:34<00:00, 121.90s/it]


# Artifact detection and labeling of Sleep Data 

We then use the pre-processed polysomnography data from above and utilize a Riemanian geometry based algorithm to detect and label artifacts contained therein. The algorithm is based on the following papers: https://hal.archives-ouvertes.fr/hal-00781701 & https://hal.science/hal-02015909 and is implemented in the yasa toolbox. This so-called "Riemannian Potato" is a clustering method that iteratively estimates the centroid of clean signal by rejecting every trial that is too far from it, thus giving you a label for each given 4 second epoch.

## 3. Process & save update epochs and hypnograms

In [5]:
# Obtain list of unique recordings
processed_files = list(set(["-".join(f.split('-')[0:3]) for f in os.listdir(save_path)]))

# Iterate over all files and process them
for idx, file in enumerate(tqdm(processed_files)):
    print(f'Detecting and labeling artifacts in file : {file}')
    # Load the data and hypnogram files
    raw = mne.io.read_raw(save_path + file + '-raw.fif.gz', preload=True) # type: ignore
    epochs = mne.read_epochs(save_path + file + '-epo.fif.gz', preload=True) # type: ignore
    hypnogram = np.load(save_path + file + '-hypnogram.npy')
    # Get sampling frequency
    sf = raw.info['sfreq']
    # Get data
    data = raw.get_data() * 1e6
    # Unravel hypnogram to match data length
    hypnogram_unravel = yasa.hypno_upsample_to_data(hypno=hypnogram, sf_hypno=1/30, 
                                                    data=data, sf_data=sf)
    # Label artifacts based on Riemannian Potato clustering algorithm
    window = 4
    art, zscores = yasa.art_detect(data, sf=sf, window=window, hypno=hypnogram_unravel, 
                                   include=(1, 2, 3, 4), method='covar', threshold=3)
    sf_art = 1 / window
    
    # Upsample art to match data length
    art_up = yasa.hypno_upsample_to_data(art, sf_art, data, sf)

    # Add -1 to hypnogram where artifacts were detected
    hypno_with_art = hypnogram_unravel.copy()
    hypno_with_art[art_up] = -1

    # Plot and save the spectrogram with the updated hypnogram
    fig = yasa.plot_spectrogram(data[1, :], sf, hypno_with_art)
    fig.savefig(fig_path + file + '-hypno-artifacts.png', dpi=300)
    plt.close(fig)

    # Downsample the hypnogram to match the number of epochs
    downsampled_hypnogram = downsample_hypnogram(hypno_with_art, raw, sf=sf, epoch_len_sec=4)

    # Create a DataFrame from the downsampled hypnogram
    metadata = pd.DataFrame({'SleepStage': downsampled_hypnogram})
    
    # Add the metadata to the epochs
    epochs.metadata = metadata

    # Save the updated hypnogram sampled at sf (256 Hz)
    np.save(save_path + file + '-hypnogram_with_art.npy', hypno_with_art)

    # Save the epochs 
    epochs.save(save_path + file + '-epo.fif.gz', overwrite=True) # type: ignore

    # Delete some objects to free up memory
    del raw, epochs, hypnogram, hypnogram_unravel, art, zscores, art_up, hypno_with_art, downsampled_hypnogram, metadata

  0%|          | 0/50 [00:00<?, ?it/s]

Detecting and labeling artifacts in file : cfs-visit5-801323


30-Aug-23 19:13:16 | WARNING | Hypnogram is SHORTER than data by 2.00 seconds. Padding hypnogram with last value to match data.size.
  2%|▏         | 1/50 [01:39<1:21:21, 99.61s/it]

Detecting and labeling artifacts in file : cfs-visit5-802739


30-Aug-23 19:15:05 | WARNING | Hypnogram is SHORTER than data by 2.00 seconds. Padding hypnogram with last value to match data.size.
  4%|▍         | 2/50 [03:33<1:26:13, 107.78s/it]

...

30-Aug-23 20:41:44 | WARNING | Hypnogram is SHORTER than data by 2.00 seconds. Padding hypnogram with last value to match data.size.
 96%|█████████▌| 48/50 [1:30:15<03:58, 119.45s/it]

Detecting and labeling artifacts in file : cfs-visit5-802635


 98%|█████████▊| 49/50 [1:32:05<01:56, 116.74s/it]

Detecting and labeling artifacts in file : cfs-visit5-801380


30-Aug-23 20:45:30 | WARNING | Hypnogram is SHORTER than data by 2.00 seconds. Padding hypnogram with last value to match data.size.
100%|██████████| 50/50 [1:33:57<00:00, 112.74s/it]


In [6]:
## Miscellanous
# import pandas as pd
# pd.Series(hypno_with_art).value_counts(normalize=True)
# yasa.sleep_statistics(hypno_with_art, sf_hyp=sf)